In [283]:
import time

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
from riotwatcher import LolWatcher, ApiError
import pprint as pp

%config IPCompleter.use_jedi=False

In [367]:
# API key
api_key = 'RGAPI-532bd4db-97d2-45a7-ac20-b7566c811150'

In [370]:
lol_watcher = LolWatcher(api_key)

In [371]:
me = lol_watcher.summoner.by_name('NA1', 'Mommys Daddy')

In [372]:
print(me)

{'id': 'R3cUwNCKn0RjY4IAzSKLB51KI3NKPrgo8rW2Wvar9UO3yOE', 'accountId': '_HK7LVlJO4KFzwAZ7hcMEiwEr3SXSlKw_10oGfYOrUFpLqM', 'puuid': '3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4Kl1NncBYV7g9tQqQJuDBJN4yPWs3tlc6Q', 'name': 'Mommys Daddy', 'profileIconId': 5141, 'revisionDate': 1647058341000, 'summonerLevel': 194}


In [373]:
puuid = me['puuid']

In [374]:
my_ranked_stats = lol_watcher.league.by_summoner('NA1', me['id'])
print(my_ranked_stats)

[{'leagueId': '802c4241-295f-4a8b-b575-90f4431fe097', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'SILVER', 'rank': 'III', 'summonerId': 'R3cUwNCKn0RjY4IAzSKLB51KI3NKPrgo8rW2Wvar9UO3yOE', 'summonerName': 'Mommys Daddy', 'leaguePoints': 75, 'wins': 18, 'losses': 9, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}, {'leagueId': '67ad7185-adb8-4921-a031-460dff9e8c2a', 'queueType': 'RANKED_FLEX_SR', 'tier': 'SILVER', 'rank': 'III', 'summonerId': 'R3cUwNCKn0RjY4IAzSKLB51KI3NKPrgo8rW2Wvar9UO3yOE', 'summonerName': 'Mommys Daddy', 'leaguePoints': 82, 'wins': 17, 'losses': 22, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


In [69]:
dicy = pd.json_normalize(my_ranked_stats)
dicy

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,802c4241-295f-4a8b-b575-90f4431fe097,RANKED_SOLO_5x5,SILVER,III,R3cUwNCKn0RjY4IAzSKLB51KI3NKPrgo8rW2Wvar9UO3yOE,Mommys Daddy,75,18,9,False,False,False,False
1,67ad7185-adb8-4921-a031-460dff9e8c2a,RANKED_FLEX_SR,SILVER,III,R3cUwNCKn0RjY4IAzSKLB51KI3NKPrgo8rW2Wvar9UO3yOE,Mommys Daddy,82,17,22,False,False,False,False


In [375]:
summoner_info = lol_watcher.summoner.by_id('NA1', 'R3cUwNCKn0RjY4IAzSKLB51KI3NKPrgo8rW2Wvar9UO3yOE')

In [376]:
dicy_df = pd.DataFrame({'summonerId': dicy[

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_22048/2542318397.py, line 1)

# Questions
* 1: Who do I play with the most?
* 2: Who do I win with the most?

In [377]:
# past 100 matches
matches = lol_watcher.match.matchlist_by_puuid(region='AMERICAS', puuid=puuid, count=100)

In [187]:
# who have i played with the most over the last 100 matches?

ls = []
for match in matches:
    match_detail = lol_watcher.match.timeline_by_match(region='AMERICAS', match_id = match)
    dic = {}
    dic['matchId'] = match
    dic['participants'] = match_detail['metadata']['participants']
    # dic['participants'] = row['metadata']['participants']
    ls.append(dic)
        
df = pd.DataFrame(ls)

In [191]:
# let's get frequency of participants over last 20 games
# first I need to unnest participants column
df_copy = df.copy()
df_copy = df_copy.explode('participants')

In [192]:
# get count of participants by participant id
df_copy['frequency'] = df_copy['participants'].map(df_copy['participants'].value_counts())

# df['Count_Column'] = df['Color'].map(df['Color'].value_counts())

In [193]:
df_copy.drop_duplicates(['participants'], inplace=True)

In [194]:
df_copy

,matchId,participants,frequency
0,NA1_4244356230,3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4K...,100
0,NA1_4244356230,H1fcBMBZ3TC4CjLTs8dkANn2rW1H271HS0c87_3A-XVZfD...,12
0,NA1_4244356230,k6HQLh3Q7bBSEVqFTdEGDbDmtCrcPHfFVpmWYEOJdBGFRL...,50
0,NA1_4244356230,7f5ZaSIrOEyQn-fMoYuM7FI404r1Au8uKupwNSeFQzt0_m...,25
0,NA1_4244356230,u8skmW61H6iRx_J4TjEq527RL8dH1vZJdGL-pPJqvtPdYf...,24
...,...,...,...
99,NA1_4206588323,8L2YueVakTARRQKbz5PsMbZucROuMwnlSmU2X_iYBeYPo1...,1
99,NA1_4206588323,f90WWjdeB8Sarohh6Jpq7Y6DymmXzaxgOVh4TjjXBoIMvc...,1
99,NA1_4206588323,Y2LWvBAAN_bB4FgbHfO5zYUI6so1vD2W5AdVOG4ddI7jMm...,1
99,NA1_4206588323,i5PDaNs3q3TkMgCqxnaagpQMdgYr5AG7TA88uTgylx5ygH...,1


In [161]:
participant = df_copy['participants'].values[1]
temp_ls = []
temp_dic = {}

summoner_info = lol_watcher.summoner.by_puuid('NA1', participant)
name = {'name':summoner_info['name']}
id = {'id':summoner_info['puuid']}
temp_dic.update(name)
temp_dic.update(id)
# temp_ls.append(temp_dic)

# temp_dic

# df = pd.DataFrame(temp_ls)

In [162]:
temp_dic

{'name': 'MissFortune0nly',
 'id': 'H1fcBMBZ3TC4CjLTs8dkANn2rW1H271HS0c87_3A-XVZfDPUvq_9V64Jv_J1x1h88zgW2mV2fg6rrw'}

In [150]:
df

,name,id
0,Mommys Daddy,3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4K...


In [129]:
participantdf

'3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4Kl1NncBYV7g9tQqQJuDBJN4yPWs3tlc6Q'

In [195]:
# now let's add in participant names on df_copy so we can know who the participants are
# lets put id and name into it's own df then merge on participants df on participant id
set_participants = set(df_copy['participants'].values)
temp_dic = {}
names_ls = []
ids_ls = []
df_summoner_info = pd.DataFrame()
for id in set_participants:
    summoner_info = lol_watcher.summoner.by_puuid('NA1', id)
    names_ls.append(summoner_info['name'])
    ids_ls.append(summoner_info['puuid'])
    # name = pd.DataFrame({'name': summoner_info['name']})
    # id = pd.DataFrame({'id': summoner_info['puuid']})
    # df_summoner_info.update(name)
    # df_summoner_info.update(id)
        
# df = pd.DataFrame(ls)
# print(temp_dic)

In [196]:
final_dic = zip(names_ls,ids_ls)
final_dic

In [197]:
df = pd.DataFrame(final_dic)

In [198]:
# rename columns
new_cols = {
0: 'name',
1: 'id'
}

df.rename(columns=new_cols, inplace=True)

In [203]:
df.head()

,name,id
0,jhínitals,pzeMvX7ssjL9mAgN51GaKf6OwFqMHNrg6tnWd2nNCy6VEc...
1,madi orbwalks,cYbdY5w0JucyqcueK4gWnmPAWRnl-X31zeBxsdp_J-D38M...
2,Daï,G204epg3SzLDfQRQIPlIy_IhnhlYTN72HKMrEvOWpJoLHs...
3,JustDyl,_zyS8kwPEkZOpivObTVoiLK2TqQmH-wvJrBLJ6LDuEYGJq...
4,POG Chip,pYns884Tdt4zzCJp780KKKUbx0gOemj9FcZj-_Y71MMFpB...


In [204]:
df_copy.head()

,matchId,participants,frequency
0,NA1_4244356230,3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4K...,100
0,NA1_4244356230,H1fcBMBZ3TC4CjLTs8dkANn2rW1H271HS0c87_3A-XVZfD...,12
0,NA1_4244356230,k6HQLh3Q7bBSEVqFTdEGDbDmtCrcPHfFVpmWYEOJdBGFRL...,50
0,NA1_4244356230,7f5ZaSIrOEyQn-fMoYuM7FI404r1Au8uKupwNSeFQzt0_m...,25
0,NA1_4244356230,u8skmW61H6iRx_J4TjEq527RL8dH1vZJdGL-pPJqvtPdYf...,24


In [208]:
# now join back to original dataframe
df_copy = df_copy.merge(right=df, how='inner', left_on=['participants'], right_on=['id'])

In [209]:
df_copy.head()

,matchId,participants,frequency,name,id
0,NA1_4244356230,3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4K...,100,Mommys Daddy,3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4K...
1,NA1_4244356230,H1fcBMBZ3TC4CjLTs8dkANn2rW1H271HS0c87_3A-XVZfD...,12,MissFortune0nly,H1fcBMBZ3TC4CjLTs8dkANn2rW1H271HS0c87_3A-XVZfD...
2,NA1_4244356230,k6HQLh3Q7bBSEVqFTdEGDbDmtCrcPHfFVpmWYEOJdBGFRL...,50,Cannøn,k6HQLh3Q7bBSEVqFTdEGDbDmtCrcPHfFVpmWYEOJdBGFRL...
3,NA1_4244356230,7f5ZaSIrOEyQn-fMoYuM7FI404r1Au8uKupwNSeFQzt0_m...,25,Velkoz ADC Only,7f5ZaSIrOEyQn-fMoYuM7FI404r1Au8uKupwNSeFQzt0_m...
4,NA1_4244356230,u8skmW61H6iRx_J4TjEq527RL8dH1vZJdGL-pPJqvtPdYf...,24,Mr Don,u8skmW61H6iRx_J4TjEq527RL8dH1vZJdGL-pPJqvtPdYf...


In [211]:
# let's drop the unneeded column
df_copy.drop('participants', axis=1, inplace=True)

In [212]:
df_copy.head()

,matchId,frequency,name,id
0,NA1_4244356230,100,Mommys Daddy,3zjuMzMzT0asfwarCbHwXwLx7b754EAMH6CCRyaNGjmW4K...
1,NA1_4244356230,12,MissFortune0nly,H1fcBMBZ3TC4CjLTs8dkANn2rW1H271HS0c87_3A-XVZfD...
2,NA1_4244356230,50,Cannøn,k6HQLh3Q7bBSEVqFTdEGDbDmtCrcPHfFVpmWYEOJdBGFRL...
3,NA1_4244356230,25,Velkoz ADC Only,7f5ZaSIrOEyQn-fMoYuM7FI404r1Au8uKupwNSeFQzt0_m...
4,NA1_4244356230,24,Mr Don,u8skmW61H6iRx_J4TjEq527RL8dH1vZJdGL-pPJqvtPdYf...


In [388]:
df_copy.value_counts()

matchId         frequency  name             id                                                                            
NA1_4206588323  1          Bench One Trick  Y2LWvBAAN_bB4FgbHfO5zYUI6so1vD2W5AdVOG4ddI7jMmp9vCUkzxNKXU0E0mF-PC-MJdjPB2i1cA    1
NA1_4215664672  1          Bluntscrank      L5wcOasar-x7MSBZHg88SILQarHxCdjMDCmDNier5Xm7erLcTxDbQdScd2Ud-EvQ6TktJk4yY0SYiA    1
NA1_4216678398  1          XREAM            Ff76hg96Cmtf-Ldmj99bD4Hygo3sXC4ADC9fcyZbvxgWT3Pb9G02gDPHA2miCWYvBtz4jps5t8PfvA    1
                           Xanel            TcD0xnOq1wFlZc27fKnamhPGMtFnktcO7RQtWXhtRu0Ov5oxTx8aQF7evBexGaSEcNrimOdn_bC77A    1
                           Yåñg             nCpzSk5zwG_OsxBsBDVCjxmBlVXcPkoVP6lyB-EVD_ZO0LBvhC9Mdc4Y7UkERPIko9vLe4_OYslblg    1
                                                                                                                             ..
NA1_4210490359  1          TheStewPig       fGrTYBhWUMYMf05RBaCT5bBPYoUV6IQ1wCzpHUsBDlXgLHPJru_RcO0EIAVR0ZamO

# Match Specific Data

In [266]:
# let's find the win column

In [252]:
match_info = lol_watcher.match.by_id('AMERICAS', matches[0])

In [256]:
match_info.keys()

dict_keys(['metadata', 'info'])

In [357]:
match_info['info'].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [382]:
print(match_info['info']['gameId'])

4244356230


In [384]:
matches[0]

'NA1_4244356230'

In [ ]:
# it looks like the match_id from match detail response is what occurs after the 'NA1_' in the matches list.
# so I'll have to join these two on a regex condition

In [353]:
# fetch last match detail
last_match = matches[0]
match_detail = lol_watcher.match.by_id(region='AMERICAS', match_id=last_match)

participants = []
for row in match_detail['info']['participants']:
    print(row.keys())
    
    #     participants_row = {}
#     participants_row['champion'] = row['championId']
#     participants_row['spell1'] = row['spell1Casts']
#     participants_row['spell2'] = row['spell2Casts']
#     participants_row['win'] = row['stats']['win']
#     participants_row['kills'] = row['stats']['kills']
#     participants_row['deaths'] = row['stats']['deaths']
#     participants_row['assists'] = row['stats']['assists']
#     participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
#     participants_row['goldEarned'] = row['stats']['goldEarned']
#     participants_row['champLevel'] = row['stats']['champLevel']
#     participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
#     participants_row['item0'] = row['stats']['item0']
#     participants_row['item1'] = row['stats']['item1']
#     participants.append(participants_row)
# df = pd.DataFrame(participants)
# df

TypeError: 'int' object is not iterable

In [340]:
# i found the win column, but these other columns look good too. let's toss it all into one dataframe and see what's up
region = 'AMERICAS'
# fetch last match detail
match = matches[0]
match_detail = lol_watcher.match.by_id(region=region, match_id=match)
row = match_detail['info']['participants'][0]
participants = []

for i in range(len(matches)):
    match_id = matches[i]
    match_detail = lol_watcher.match.by_id(region=region, match_id=match_id)
    for row in match_detail['info']['participants']:
        participants_row['assists'] = row['assists']
        # participants_row = {}
        # participants_row['champion'] = row['championId']
        # participants_row['spell1'] = row['spell1Casts']
        # participants_row['spell2'] = row['spell2Casts']
        # participants_row['win'] = row['win']
        # participants_row['kills'] = row['kills']
        # participants_row['deaths'] = row['deaths']
        # participants_row['assists'] = row['assists']
        # participants_row['totalDamageDealt'] = row['totalDamageDealt']
        # participants_row['goldEarned'] = row['goldEarned']
        # participants_row['champLevel'] = row['champLevel']
        # participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
        # participants_row['item0'] = row['item0']
        # participants_row['item1'] = row['item1']
        participants.append(participants_row)

#     for row in ls['
# for ls in match_detail['info']['participants'][:]:
#     for row in ls.items():

        
    
        # participants_row[f'{k}'] = v
    
df = pd.DataFrame(participants)
df

,assists
0,27
1,27
2,27
3,27
4,27
...,...
995,27
996,27
997,27
998,27


In [380]:
# i found the win column, but these other columns look good too. let's toss it all into one dataframe and see what's up
# need to add match_id in here to index this dataframe
region = 'AMERICAS'
# fetch last match detail
match = matches[0]
match_detail = lol_watcher.match.by_id(region=region, match_id=match)
row = match_detail['info']['participants'][0]

participants = []
participants_row = {}

participants_row['assists'] = row['assists']
# participants_row = {}
# participants_row['champion'] = row['championId']
# participants_row['spell1'] = row['spell1Casts']
# participants_row['spell2'] = row['spell2Casts']
# participants_row['win'] = row['win']
# participants_row['kills'] = row['kills']
# participants_row['deaths'] = row['deaths']
# participants_row['assists'] = row['assists']
# participants_row['totalDamageDealt'] = row['totalDamageDealt']
# participants_row['goldEarned'] = row['goldEarned']
# participants_row['champLevel'] = row['champLevel']
# participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
# participants_row['item0'] = row['item0']
# participants_row['item1'] = row['item1']
participants.append(participants_row)

#     for row in ls['
# for ls in match_detail['info']['participants'][:]:
#     for row in ls.items():



# participants_row[f'{k}'] = v

df = pd.DataFrame(participants)
df

,assists
0,9


In [330]:
# lets make a list of the keys and plug that in
keys_ls = []
vals_ls = []
for ls in match_detail['info']['participants'][0:2]:
    for k,v in ls.items():
        keys_ls.append(k)
        vals_ls.append(v)
    
dic = dict(zip(keys_ls,vals_ls))

In [318]:
df = pd.DataFrame(dic)

In [320]:
df

,assists,baronKills,bountyLevel,challenges,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,...,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
12AssistStreakCount,4,0,2,0.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
abilityUses,4,0,2,213.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
acesBefore15Minutes,4,0,2,0.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
alliedJungleMonsterKills,4,0,2,0.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
baronTakedowns,4,0,2,0.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wardTakedowns,4,0,2,1.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
wardTakedownsBefore20M,4,0,2,1.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
wardsGuarded,4,0,2,0.0,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True
statPerks,4,0,2,NaN,15479,16,83,Yorick,0,2,...,3698,3,6,1,0,18,1,1,5,True


In [305]:
participants_row

{'win': True}

In [297]:
# for i in range(match
pp.pprint(match_detail['info']['participants'][0].keys())

dict_keys(['assists', 'baronKills', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'goldEarned', 'goldSpent', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'objectivesStolen', 'objectivesStolenAssists', 'participantId', 'pentaKills', '

# Summoner Specific Data